# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [27]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import warnings
warnings.filterwarnings('ignore')

# Import API key
from api_keys import geoapify_key

In [28]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,new norfolk,-42.7826,147.0587,8.29,95,100,1.42,AU,1730227385
1,1,adamstown,-25.0660,-130.1015,20.84,73,4,2.11,PN,1730227385
2,2,atafu village,-8.5421,-172.5159,27.79,77,22,5.76,TK,1730227386
3,3,cole harbour,44.6724,-63.4751,9.28,41,20,5.66,CA,1730227386
4,4,howard springs,-12.4970,131.0470,27.95,91,100,2.99,AU,1730227386


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [29]:
%%capture --no-display

# Create the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    # Configure the map plot
    geo="True", # show geo data
    tiles="OSM",  # OpenStreetMap tiles
    frame_width=700,
    frame_height=500,
    size='Humidity',  # Size of the points determined by Humidity
    color="City",  # Color by city
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [30]:
# Narrow down cities that fit criteria and drop any results with null values
# I am going to seek out cold places that have little wind and fairly clear skies
# Filter for temp between -5 and 10 degrees C.
filter1 = city_data_df.loc[(city_data_df['Max Temp'] > -5) & (city_data_df['Max Temp']< 10)]

# Filter for wind below 5
filter2 = filter1.loc[filter1['Wind Speed'] < 5]

# Filter for cloudiness below 25
filter3 = filter2.loc[filter2['Cloudiness'] < 25]

# Drop any rows with null values
clean_filtered = filter3.dropna().reset_index()

# Display sample data
clean_filtered

,index,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,23,23,hami,42.8000,93.4500,9.44,64,3,2.47,CN,1730227388
1,47,47,murray bridge,-35.1167,139.2667,9.18,89,0,0.39,AU,1730227390
2,78,78,ashgabat,37.9500,58.3833,8.92,87,0,1.03,TM,1730227393
3,101,101,hammerfest,70.6634,23.6821,1.25,64,20,4.63,NO,1730227396
4,162,162,nuuk,64.1835,-51.7216,-3.66,80,20,3.09,GL,1730227150
5,183,183,ulan bator,47.9077,106.8832,-0.93,86,0,1.00,MN,1730227403
6,192,192,pechory,57.8164,27.6119,3.29,83,24,2.68,RU,1730227404
7,209,209,khovd,48.0056,91.6419,0.47,45,10,2.22,MN,1730227406
8,259,259,christchurch,-43.5333,172.6333,9.07,93,0,1.54,NZ,1730227410
9,263,263,mil'kovo,55.6078,37.7956,4.79,49,5,3.59,RU,1730227411


### Step 3: Create a new DataFrame called `hotel_df`.

In [31]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = clean_filtered[['City', 'Country', 'Lat', 'Lng', 'Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,hami,CN,42.8000,93.4500,64,
1,murray bridge,AU,-35.1167,139.2667,89,
2,ashgabat,TM,37.9500,58.3833,87,
3,hammerfest,NO,70.6634,23.6821,64,
4,nuuk,GL,64.1835,-51.7216,80,
5,ulan bator,MN,47.9077,106.8832,86,
6,pechory,RU,57.8164,27.6119,83,
7,khovd,MN,48.0056,91.6419,45,
8,christchurch,NZ,-43.5333,172.6333,93,
9,mil'kovo,RU,55.6078,37.7956,49,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [32]:

# Set parameters to search for a hotel
limit = 1
radius_meters = 10000
params = {
    'categories': 'accommodation.hotel',
    'apiKey': geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']
    
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{longitude},{latitude},{radius_meters}'
    params["bias"] = f'proximity:{longitude},{latitude}'

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
hami - nearest hotel: Хами
murray bridge - nearest hotel: Bridgeport
ashgabat - nearest hotel: Kuwwat Myhmanhanasy
hammerfest - nearest hotel: Thon Hotel Hammerfest
nuuk - nearest hotel: Nordbo
ulan bator - nearest hotel: LION Hotel
pechory - nearest hotel: Печоры Парк
khovd - nearest hotel: Khovd hotel
christchurch - nearest hotel: Ibis Hotel Christchurch
mil'kovo - nearest hotel: Дрозды
nogliki - nearest hotel: Ноглики
zanjan - nearest hotel: هتل قصر
prince george - nearest hotel: Travelodge by Wyndham Prince George
kittila - nearest hotel: Hotelli Kittilä
qarauyl - nearest hotel: Гомтиница Ер-нур
yunak - nearest hotel: No hotel found
bohuslav - nearest hotel: No hotel found
fryanovo - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
0,hami,CN,42.8000,93.4500,64,Хами
1,murray bridge,AU,-35.1167,139.2667,89,Bridgeport
2,ashgabat,TM,37.9500,58.3833,87,Kuwwat Myhmanhanasy
3,hammerfest,NO,70.6634,23.6821,64,Thon Hotel Hammerfest
4,nuuk,GL,64.1835,-51.7216,80,Nordbo
5,ulan bator,MN,47.9077,106.8832,86,LION Hotel
6,pechory,RU,57.8164,27.6119,83,Печоры Парк
7,khovd,MN,48.0056,91.6419,45,Khovd hotel
8,christchurch,NZ,-43.5333,172.6333,93,Ibis Hotel Christchurch
9,mil'kovo,RU,55.6078,37.7956,49,Дрозды


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [33]:
# Remove cities without a hotel
hotel_df = hotel_df.loc[hotel_df['Hotel Name'] != 'No hotel found'].reset_index().drop(columns='index')
hotel_df    

,City,Country,Lat,Lng,Humidity,Hotel Name
0,hami,CN,42.8000,93.4500,64,Хами
1,murray bridge,AU,-35.1167,139.2667,89,Bridgeport
2,ashgabat,TM,37.9500,58.3833,87,Kuwwat Myhmanhanasy
3,hammerfest,NO,70.6634,23.6821,64,Thon Hotel Hammerfest
4,nuuk,GL,64.1835,-51.7216,80,Nordbo
5,ulan bator,MN,47.9077,106.8832,86,LION Hotel
6,pechory,RU,57.8164,27.6119,83,Печоры Парк
7,khovd,MN,48.0056,91.6419,45,Khovd hotel
8,christchurch,NZ,-43.5333,172.6333,93,Ibis Hotel Christchurch
9,mil'kovo,RU,55.6078,37.7956,49,Дрозды


In [34]:
%%capture --no-display

# Create the map plot
map_plot2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    # Configure the map plot
    geo="True", # show geo data
    tiles="OSM",  # OpenStreetMap tiles
    frame_width=700,
    frame_height=500,
    size='Humidity',  # Size of the points determined by Humidity
    color="City",  # Color by city
    scale=1.5, # the humidity is low in these cities, so I increased the scale of the points
    hover_cols=['Hotel Name', 'Country'] # added hotel names and country code to the hover
)

# Display the map
map_plot2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)